In [ ]:
import argparse
import shutil
import errno
import time
import json
import glob
import os
import cv2
import math

In [ ]:
#置信度阈值
confidence_threshold = 0.98
# 输出图片的固定高度
fixed_height = 48

In [ ]:

def create_label(json_file, image_file, output_dir):

#         print("----------      {}     ".format(json_file))
    if not os.path.exists(json_file) or not os.path.exists(image_file):
        print('【警告】文件不存在  --------file:  {} '.format(json_file))
        #print(image_file)
        #print(json_file)
        return

    with open(json_file, 'r', encoding='utf8')as fp:
        json_data = json.load(fp)

    bg_image = cv2.imread(image_file)


    image_height = bg_image.shape[0]
    image_width = bg_image.shape[1]
    print('{}  width={} height={}'.format(image_file, image_width, image_height))

    new_lines = ''

    for index, item in enumerate(json_data['Blocks']):
        if item['BlockType'] != "WORD" or  item['Confidence']< confidence_threshold  :
            continue

        points = item['Geometry']['Polygon'] 
        #print(points)
        left = int(points[0]['X'] * image_width)

        if int(points[1]['X'] * image_width ) < left:
            left = int(points[1]['X'] * image_width)

        top = int(points[0]['Y'] * image_height)

        if int(points[1]['Y'] * image_height) < top:
            size = bg_image.shape
            top = int(points[1]['Y'] * image_height) 


        width = int(abs(points[1]['X']  - points[0]['X']) * image_width)
        height = int(abs(points[3]['Y']  - points[0]['Y']) * image_height)
        text = item['Text'].lstrip().rstrip()




        c_img = bg_image[int(top): int(top + height), int(left): int(left + width)]
        sub_image_name = image_file.split('/')[-1].replace('.', '_')+'_'+str(index).zfill(5)+".jpg"

        if c_img.shape[0] < 10 or c_img.shape[1] < 10:
            continue

        scale = fixed_height / height
        new_w, new_h = int(width * scale), fixed_height
        resize_img = cv2.resize(c_img, (new_w, new_h))
        print("top={} left={} width={} height={} text:[{}]  file: {} ".format(top, left, new_w, new_h, text, sub_image_name))
        cv2.imwrite(os.path.join(output_dir,  sub_image_name), resize_img)

        line = "{} {}\n".format(sub_image_name, text)
        new_lines += line


    return new_lines

In [ ]:
def parse_file_list(input_dir, output_dir):
    """
    """

    #print(" input dir: {} ".format(input_dir))
    dirs = os.listdir(input_dir)
    types = ('*.jpg', '*.png', '*.jpeg', '*.JPG', '*.PNG', '*.JPEG')
    files_grabbed = []
    for files in types:
        files_grabbed.extend(glob.glob(os.path.join(input_dir, files)))

    files_grabbed.sort()
    total_lines = ''
    total_count = 0
    for index, file in enumerate(files_grabbed):
        json_file = os.path.join(input_dir, file.split('/')[-1].split('.')[0] + '.json')
        total_count += 1
        
        if total_count>2:
            break
        if os.path.exists(json_file):
            new_lines = create_label(json_file, file, output_dir)
            total_lines += new_lines
        else:
            print("  {} 不存在 ".format(json_file))

    gt_labels_file = os.path.join(output_dir, 'label.txt')
    with open(gt_labels_file, "w") as f:
        f.write(total_lines)
        print('【输出】生成 lambelme 格式文件  输出路径{}, 文件个数 {} , 对象个数 {}.'.format(gt_labels_file, len(files_grabbed), len(total_lines)))

In [ ]:
"""
input_dir   图片和识别后的json 文件 保存路径， 在一个文件夹下， json 是textract 格式
output_dir   输出格式
"""

input_dir = '../ocr_data/001/image'
output_dir = '../ocr_data/001/output'



if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)


parse_file_list(input_dir, output_dir )